Notebook specific for others ( sockets + lighting ) model improvement

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from skforecast.model_selection import backtesting_forecaster
from skforecast.ForecasterAutoreg import ForecasterAutoreg

import plotly
import plotly.graph_objects as go


from astral import LocationInfo
from astral.sun import sun


In [2]:
from custom_utils import *

In [3]:
def get_raw_data():
    data_path = "./../data/"
    file_name = "data.csv"
    data = pd.read_csv(data_path + file_name)
    data = first_preprocess(data, data_path)
    return data

In [4]:
data = get_raw_data()
data.head()

d:\RA_work\timeseries_data\notebooks\custom_utils.py:85: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data[column] = data[column].interpolate(method="linear", limit_direction="both")


,space_heating,hot_water,sockets,lighting,bld_engcons,car_chargers,weekend,bank_holiday,day_of_week,day_of_month,...,forecast_weathertype,forecast_windspeed,forecast_uvindex,forecast_precipitationprobability,forecast_visibility,week_of_year,sine_hour,cos_hour,sine_forecast_winddirection,cos_forecast_winddirection
datetime,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00+00:00,3.425193,0.000000,0.049433,0.154873,3.629499,0.0,0,1,5,1,...,2.0,11.0,0.0,0.0,5,53,0.000000,1.000000,0.000000,1.00000
2021-01-01 01:00:00+00:00,4.012907,0.000000,0.047753,0.154846,4.215506,0.0,0,1,5,1,...,2.0,11.0,0.0,0.0,5,53,0.258819,0.965926,0.000000,1.00000
2021-01-01 02:00:00+00:00,5.342417,0.444782,0.051869,0.154997,5.994066,0.0,0,1,5,1,...,2.0,11.0,0.0,1.0,5,53,0.500000,0.866025,0.000000,1.00000
2021-01-01 03:00:00+00:00,5.756598,0.000000,0.043028,0.154626,5.954252,0.0,0,1,5,1,...,0.0,11.0,0.0,1.0,5,53,0.707107,0.707107,-0.382683,0.92388
2021-01-01 04:00:00+00:00,5.415492,0.000000,0.050306,0.154498,5.620296,0.0,0,1,5,1,...,0.0,11.0,0.0,1.0,5,53,0.866025,0.500000,-0.382683,0.92388


In [5]:
exo_columns = ['forecast_temperature', 'forecast_feelslike', 'forecast_weathertype',
       # 'forecast_windspeed', 'forecast_uvindex',
       # 'forecast_precipitationprobability', 'forecast_visibility',
       # 'week_of_year', 'daylight_hours', 'is_daylight', 
       'sine_hour',
       'cos_hour', 
       # 'sine_month', 'cos_month', 'sine_day_of_week',
       # 'cos_day_of_week', 'sine_forecast_winddirection',
       # 'cos_forecast_winddirection', 'sine_sunrise_hour', 'cos_sunrise_hour',
       # 'sine_sunset_hour', 'cos_sunset_hour'] 
]
endo_columns = ['weekend', "month", "day_of_week" ] # 'bank_holiday'] # 'day_of_month', 'year']
target_column = ["y"]
column_name = exo_columns + endo_columns  + target_column
test_date = '2023-01-01'

# transform the data
others_data = data.copy()
# "hot_water", "sockets", "lighting"
others_data["others"] = others_data["sockets"] + others_data["lighting"] # + others_data["hot_water"]
others_data = others_data.rename(columns={"others": "y"})
# others_data["y"] = others_data["others"].shift(-1)
# others_data = others_data.dropna(axis=0)

# others_data_scaler, others_transformed_data = create_std_scaler(others_data, column_name[:-1])
# others_target_scaler, others_transformed_target_data = create_std_scaler(others_data, column_name[-1])

# others_transformed_data = pd.merge(others_transformed_data, others_transformed_target_data,
#                               left_index=True,
#                               right_index=True)

# # Training and testing data for comms and services
#train_data = others_transformed_data[others_transformed_data.index < test_date][column_name]
#test_data = others_transformed_data[others_transformed_data.index >= test_date][column_name]
#train_data.shape, test_data.shape
# adding baseline minimum of y
others_data.loc[others_data["y"] <= 0, "y"] = others_data[others_data["y"] > 0]["y"].min()

others_train_data = others_data[others_data.index < test_date][column_name]
others_test_data = others_data[others_data.index >= test_date][column_name]

In [6]:
others_main_data = pd.concat([others_train_data, others_test_data], axis=0)
others_main_data.shape

(26280, 9)

In [7]:
# params = {
#     'n_estimators': 100,
#     'criterion':  'absolute_error',
#     'max_depth': 2,
#     'random_state': 994564,
#     "max_features": "log2",
#     'verbose': 0
# }
# forecaster = ForecasterAutoreg(
#     regressor=RandomForestRegressor(**params), 
#     lags=24
# )
forecaster = ForecasterAutoreg(
    # regressor=RandomForestRegressor(),
    regressor=LinearRegression(),
    lags=2
)
# forecaster = ForecasterAutoreg(
#     regressor=DecisionTreeRegressor(max_depth=18, max_features="log2"),
#     lags=6
# )
forecaster.fit(
    y=others_train_data["y"],
    exog=others_train_data[exo_columns + endo_columns]
)

In [8]:
def custom_metric(y_true, y_pred):
    '''
    Calculate the mean squared error using only the predicted values of the last
    3 months of the year.
    '''
    return calculate_smape(actual=y_true, predicted=y_pred)

In [9]:
def plot_predictions(plot_data, title_text):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=plot_data["time"].to_numpy(), y=plot_data["y"].to_numpy(),
                        mode='lines+markers',
                        name='actual'))
    fig.add_trace(go.Scatter(x=plot_data["time"].to_numpy(), y=plot_data["pred"].to_numpy(),
                        mode='lines+markers',
                        name='predictions'))
    fig.update_layout(title_text=title_text)
    fig.show()

In [10]:
metric, others_predictions_training = backtesting_forecaster(
    forecaster         = forecaster,
    y                  = others_train_data['y'],
    exog               = others_train_data[exo_columns + endo_columns],
    steps              = 1,
    metric             = custom_metric,
    initial_train_size = None,
    refit              = False,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = True
)

print(f"Backtest error: {metric:.2f}")
others_predictions_training.head()

  0%|          | 0/17518 [00:00<?, ?it/s]

Backtest error: 22.60


,pred
2021-01-01 02:00:00+00:00,0.202141
2021-01-01 03:00:00+00:00,0.216787
2021-01-01 04:00:00+00:00,0.219529
2021-01-01 05:00:00+00:00,0.236178
2021-01-01 06:00:00+00:00,0.236935


In [11]:
others_predictions_training["y"] = others_train_data["y"]
others_predictions_training = others_predictions_training.reset_index(drop=0)
others_predictions_training = others_predictions_training.rename(columns={"index": "time"})
# predictions_training[["pred", "y"]] = others_target_scaler.inverse_transform(predictions_training[["pred", "y"]])
others_predictions_training.head()

,time,pred,y
0,2021-01-01 02:00:00+00:00,0.202141,0.206867
1,2021-01-01 03:00:00+00:00,0.216787,0.197654
2,2021-01-01 04:00:00+00:00,0.219529,0.204804
3,2021-01-01 05:00:00+00:00,0.236178,0.197304
4,2021-01-01 06:00:00+00:00,0.236935,0.197220


In [12]:
plot_predictions(others_predictions_training, title_text="Others Training data")

In [13]:
metric, others_predictions = backtesting_forecaster(
    forecaster         = forecaster,
    y                  = others_main_data['y'],
    exog               = others_main_data[exo_columns + endo_columns],
    steps              = 1,
    metric             = custom_metric,
    initial_train_size = others_main_data[others_main_data.index < "2023-01-01"].shape[0],
    fixed_train_size   = True,
    refit              = 1000,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = True
)

print(f"Backtest error: {metric:.2f}")
others_predictions.head()

  0%|          | 0/8760 [00:00<?, ?it/s]

Backtest error: 21.54


,pred
2023-01-01 00:00:00+00:00,0.157499
2023-01-01 01:00:00+00:00,0.147576
2023-01-01 02:00:00+00:00,0.131442
2023-01-01 03:00:00+00:00,0.140719
2023-01-01 04:00:00+00:00,0.170219


In [14]:
others_predictions["y"] = others_main_data[others_main_data.index >= "2023-01-01"]["y"]
others_predictions = others_predictions.reset_index(drop=0)
others_predictions = others_predictions.rename(columns={"index": "time"})
# predictions[["pred", "y"]] = others_target_scaler.inverse_transform(predictions[["pred", "y"]])
others_predictions.head()

,time,pred,y
0,2023-01-01 00:00:00+00:00,0.157499,0.176759
1,2023-01-01 01:00:00+00:00,0.147576,0.142263
2,2023-01-01 02:00:00+00:00,0.131442,0.137709
3,2023-01-01 03:00:00+00:00,0.140719,0.160749
4,2023-01-01 04:00:00+00:00,0.170219,0.173842


In [15]:
plot_predictions(others_predictions, title_text="Others Test data")

In [16]:
cal_metrics(others_predictions["pred"].to_numpy(), others_predictions["y"].to_numpy())

,MAE,MAPE,R2_Score,SMAPE,nRMSE,RMSE,MASE
0,0.109352,25.53,0.600196,21.54,0.073269,0.204618,-0.11


In [17]:
cal_metrics(others_predictions_training["pred"].to_numpy(), others_predictions_training["y"].to_numpy())

,MAE,MAPE,R2_Score,SMAPE,nRMSE,RMSE,MASE
0,0.069936,24.14,0.557229,22.6,0.0432,0.15144,-0.07


In [18]:
tmp_data = pd.concat([others_predictions_training, others_predictions], axis=0)
tmp_data = tmp_data.drop(columns=["y"], axis=1)

# tmp_data1 = others_main_data[others_main_data.index < '2021-01-01 06:00:00+00:00'][["y"]].reset_index(drop=0)
# tmp_data1[["y"]] = others_target_scaler.inverse_transform(tmp_data1[["y"]])

tmp_data1 = tmp_data.rename(columns={"y": "pred", "datetime": "time"})
# tmp_data = pd.concat([tmp_data1, tmp_data], axis=0)
tmp_data = tmp_data.set_index("time")
tmp_data = tmp_data.rename(columns={"pred": "others_predictions"})

tmp_data.to_csv("others_pred.csv", index=True)
tmp_data.shape

(26278, 1)